In [13]:
import tkinter as tk
from tkinter import filedialog

# create a root window, bring to top and withdraw it
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw() 

# request the user to select the file
file_path = filedialog.askopenfilename(parent = root, title = "Select Exported Spike Recording", filetypes = [("Comma-separated value","*.csv")], initialdir = "../data/")

In [57]:
import pandas as pd
import ipywidgets as widgets

# read first line of the DF to get the channel names
df = pd.read_csv(filepath_or_buffer = file_path, nrows = 0)
channel_names = df.columns.values

# a box for time and signal
sel_time = widgets.Select(options = channel_names, disabled = False)
time_box = widgets.VBox([widgets.Label(value = "Select the time information channel: "), sel_time_channel])

sel_signal = widgets.Select(options = channel_names, disabled = False)
signal_box = widgets.VBox([widgets.Label(value = "Select the signal channel: "), sel_signal])

time_sig_box = widgets.HBox([time_box, signal_box], layout = widgets.Layout(border = "1px solid"))

# one box for the stimuli
chk_load_reg_stim = widgets.Checkbox(value = True, description = "Load regular electrical stimuli from channel: ", indent = False)
sel_reg_stimuli = widgets.Select(options = channel_names, disabled = False)
reg_stim_box = widgets.VBox([chk_load_reg_stim, sel_reg_stimuli])

chk_load_extra_stim = widgets.Checkbox(value = False, description = "Load extra electrical stimuli from channel: ", indent = False)
sel_ex_stimuli = widgets.Select(options = channel_names, disabled = False)
ex_stim_box = widgets.VBox([chk_load_extra_stim, sel_ex_stimuli])

chk_load_mech_stim = widgets.Checkbox(value = False, description = "Load mechanical stimuli from channel: ", indent = False)
mech_stimuli = widgets.Select(options = channel_names, disabled = False)
mech_stim_box = widgets.VBox([chk_load_mech_stim, mech_stimuli])

stim_box = widgets.HBox([reg_stim_box, ex_stim_box, mech_stim_box], layout = widgets.Layout(border = "1px solid"))

# one box for the AP channels
chk_load_aps = widgets.Checkbox(value = True, description = "Load action potentials from channels: ", indent = False)
sel_ap = widgets.SelectMultiple(options = channel_names, disabled = False)

ap_box = widgets.VBox([chk_load_aps, sel_ap], layout = widgets.Layout(border = "1px solid"))

# the function that is called when the button is clicked
def load_file(btn):
    print("Test!")

# button to load the file
btn_load_file = widgets.Button(description = "Import File", layout = widgets.Layout(border = "1px solid"))
btn_load_file.on_click(load_file)
display(time_sig_box, stim_box, ap_box, btn_load_file)

Button(description='Import File', layout=Layout(border='1px solid'), style=ButtonStyle())

In [ ]:
%load_ext autoreload
%autoreload 2

from importers import SpikeImporter

data_folder = "../data/"
file_name = "11_10_27U2a_2000s_2300s_20000Hz_Force_AllFibres"
file_ending = ".csv"

time_channel = "Time"
signal_channel = "1 Signal"
stimulus_channel = "32 DigMark"
ap_marker_channels = ["801 nw-1", "12 nw-1", "14 nw-1"]
force_channel = "3 Force"
# extra_stimuli_channel = "801 DigMark"

# load the animal file
importer = SpikeImporter(filepath = data_folder + file_name + file_ending, time_channel = time_channel, signal_channel = signal_channel)

print("Data loaded.")

# define the max. time so that two sequential waveform-values are belonging to the same AP
# the time is in seconds, so 0.005 equals 5ms
max_gap_time = 0.005
force_thresh = 0.5

# retrieve the events from the importer 
el_stimuli = importer.get_electrical_stimuli(regular_stimulus_channel = stimulus_channel)
ex_stimuli = importer.get_extra_stimuli(extra_stimulus_channel = extra_stimuli_channel, regular_el_stimuli = el_stimuli)
mech_stimuli = importer.get_mechanical_stimuli(force_channel = force_channel, threshold = force_thresh, max_gap_time = 0.005)
actpots = importer.get_action_potentials(max_gap_time = max_gap_time, ap_marker_channels = ap_marker_channels, \
                                       el_stimuli = el_stimuli, mech_stimuli = mech_stimuli)
raw_signal = importer.get_raw_signal_split_by_stimuli(el_stimuli = el_stimuli, verbose = False)